### 12.1 Your First Agent

In [2]:
from langchain_openai import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)

def plus(a, b):
    return a + b

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        StructuredTool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. This tool take two arguments, both should be numbers."
        ),
    ]
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_3912\2961844702.py:10: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(




> Entering new AgentExecutor chain...
Thought: I can use the Sum Calculator tool to find the total cost of all the given amounts.

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 355.39,
    "b": 924.87
  }
}
```
Observation: 1280.26
Thought:Now, I will continue adding the remaining amounts to the sum calculated so far.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 1280.26,
    "b": 721.2
  }
}
```
Observation: 2001.46
Thought:Now, I will continue adding the remaining amounts to the sum calculated so far.

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 2001.46,
    "b": 1940.29
  }
}
```
Observation: 3941.75
Thought:I will now add the remaining amounts to the sum calculated so far to find the total cost.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 3941.75,
    "b": 573.63
  }
}
```
Observation: 4515.38
Thought:I will continue adding the remaining amounts to the sum calculated so

{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost is $4616.1'}

### 12.3 Zero-shot ReAct Agent

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)

def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    tools=[
        Tool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. Use this tool by sending a pair of number separated by a comma. \n\nExample:1,2"
        ),
    ]
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...
This is a sum of multiple numbers, so I should use the Sum Calculator tool multiple times to find the total sum.
Action: Sum Calculator
Action Input: 355.39, 924.87
Observation: 1280.26
Thought:I need to continue adding the remaining numbers to the sum.
Action: Sum Calculator
Action Input: 1280.26, 721.2
Observation: 2001.46
Thought:I need to continue adding the remaining numbers to the sum.
Action: Sum Calculator
Action Input: 2001.46, 1940.29
Observation: 3941.75
Thought:I need to continue adding the remaining numbers to the sum.
Action: Sum Calculator
Action Input: 3941.75, 573.63
Observation: 4515.38
Thought:I need to continue adding the remaining numbers to the sum.
Action: Sum Calculator
Action Input: 4515.38, 65.72
Observation: 4581.1
Thought:I need to continue adding the remaining numbers to the sum.
Action: Sum Calculator
Action Input: 4581.1, 35.00
Observation: 4616.1
Thought:I need to continue adding the remaining numbers to the sum.
A

{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': '$5273.38'}

### 12.4 OpenAI Functions Agent

In [ ]:
from typing_extensions import Type
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)

class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The first number")
    b: float = Field(description="The second number")

class CalculatorTool(BaseTool):
    name: Type[str] = "CalculatorTool"
    description: Type[str] = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    args_schema : Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema
    
    def _run(self, a, b):
        return a + b

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        CalculatorTool(),
    ]
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `CalculatorTool` with `{'a': 35539, 'b': 92487}`


128026
Invoking: `CalculatorTool` with `{'a': 128026, 'b': 72120}`


200146
Invoking: `CalculatorTool` with `{'a': 200146, 'b': 194029}`


394175
Invoking: `CalculatorTool` with `{'a': 394175, 'b': 57363}`


451538
Invoking: `CalculatorTool` with `{'a': 451538, 'b': 6572}`


458110
Invoking: `CalculatorTool` with `{'a': 458110, 'b': 3500}`


461610
Invoking: `CalculatorTool` with `{'a': 461610, 'b': 55200}`


516810
Invoking: `CalculatorTool` with `{'a': 516810, 'b': 7616}`


524426
Invoking: `CalculatorTool` with `{'a': 524426, 'b': 2912}`


527338The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.

> Finished chain.


{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.'}

### 12.5 Search Tool

In [8]:
from typing_extensions import Type
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1)

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query:str = Field(description="The query you will search for")

class StockMarketSymbolSearchTool(BaseTool):
    name: Type[str] = "StockMarketSymbolSearchTool"
    description: Type[str] = """ 
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for Apple Company
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema    
    
    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
    ]
)

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare'}`




c:\Users\USER\anaconda3\envs\Fullstack-GPT\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


R2 is a major destination for Cloudflare Logs. During the primary incident window, all available resources became saturated attempting to buffer and deliver data to R2. This prevented other jobs from acquiring resources to process their queues. Data loss (dropped logs) occurred when the job queues expired their data (to allow for new, incoming ... Get the latest news on how products at Cloudflare are built, technologies used, and join the teams helping to build a better Internet. San Francisco, CA, November 7, 2024 — Cloudflare, Inc. (NYSE: NET), the leading connectivity cloud company, today announced financial results for its third quarter ended September 30, 2024. "I am pleased with our results for the third quarter—exceeding expectations for revenue, operating margin, and free cash flow while also reaching a key ... At Cloudflare, we are constantly innovating and launching new features and capabilities across our product portfolio. Today's roundup blog post shares two exciting updat

{'input': "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have.",
 'output': "The stock market symbol for Cloudflare is NET. Now, let's analyze Cloudflare's stock performance to determine if it's a potential good investment.\n\nTo analyze if Cloudflare's stock is a good investment, we need to consider various factors such as the company's financial health, growth potential, competitive position, and market trends. Here are some key points to consider:\n\n1. **Financial Performance**: Look at Cloudflare's revenue growth, profitability, and cash flow. Analyze if the company is consistently growing its revenue and improving its profitability.\n\n2. **Market Potential**: Evaluate the market potential for Cloudflare's services. Consider the demand for cloud-based services and cybersecurity solutions, which are areas where Cloudflare operates.\n\n3. **Competitive Position**: Assess Cloudflare's compet